<a href="https://colab.research.google.com/github/dominique-nshimyimana/Car-State/blob/master/car_state_action.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Car State-Action Prection 
#Colab: TensorFlow and GoogleDrive

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
%tensorflow_version 2.x

In [0]:
from google.colab import drive
mounted = '/drive'
drive.mount(mounted)

Mounted at /drive


# NN state Evaluation

In [0]:
import tensorflow as tf
import numpy as np
from os import listdir
from os.path import isfile, join
from sklearn.model_selection import train_test_split


class NeuralDynamics(tf.keras.Model):

    def __init__(self, state_size, action_size):

        super(NeuralDynamics, self).__init__()

        self.training_counter = 0

        if type(state_size) is tuple:
          self.state_size = state_size
          self.action_size = action_size
          self.out_size = state_size[1]
        else:
          self.state_size = state_size
          self.action_size = action_size
          self.out_size = state_size

        self.layer1 = tf.keras.layers.Dense(300, activation='relu', kernel_regularizer=tf.keras.regularizers.l1(0.1))
        self.dropout = tf.keras.layers.Dropout(rate=0.5)
        self.layer2 = tf.keras.layers.Dense(300, activation='relu', kernel_regularizer=tf.keras.regularizers.l1(0.1))

        self.layer_state1 = tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l1(0.1))
        self.layer_state2 = tf.keras.layers.Dense(self.out_size, kernel_regularizer=tf.keras.regularizers.l1(0.1))

        self.known_inputs = False
        self.training_step = 0

    @tf.function
    def call(self, inputs):

        state, action = inputs
        if type(self.state_size) is tuple and self.state_size[0] != state.shape[0]:
          missing_zeros = self.state_size[0]-state.shape[0]
          paddings = tf.constant([[missing_zeros, 0], [0, 0]])
          state = tf.pad(state, paddings, "CONSTANT")

        x = tf.concat((state, action), axis=1)
        x = self.layer1(x)
        x = self.dropout(x)
        x = self.layer2(x)
        x = self.dropout(x)

        x = self.layer_state1(x)
        x = self.dropout(x)
        state = self.layer_state2(x)
        return state

# Prepare Data

In [0]:
data_dir = "/drive/My Drive/neuronyte_logging"
data_files = [join(data_dir, f) for f in listdir(data_dir) if isfile(join(data_dir, f)) and ".directory" not in f]

cutoff_beginning = 300
cutoff_end = 1000

xs_states = []
xs_actions = []
ys = []

for f in data_files:
    print(f)
    data = np.loadtxt(f, delimiter=', ', skiprows=1, dtype=np.float32)[cutoff_beginning:-cutoff_end, :]
    x_states = [data[i, 1:-3] for i in range(len(data))]
    x_actions = [data[i, -3:] for i in range(len(data))]
    y = x_states[1:]
    x_states, x_actions = x_states[:-1], x_actions[:-1]
    xs_states += x_states
    xs_actions += x_actions
    ys += y

xs_states = np.vstack(xs_states)
xs_actions = np.vstack(xs_actions)
ys = np.vstack(ys)

x_states_train, x_states_test, x_actions_train, x_actions_test, y_train, y_test = train_test_split(xs_states, xs_actions, ys, test_size=0.10)

/drive/My Drive/neuronyte_logging/NeuroNyte_1585748620.316643
/drive/My Drive/neuronyte_logging/NeuroNyte_1585745385.311904


# Initialize Model

In [0]:
decay=tf.keras.optimizers.schedules.ExponentialDecay(0.0004, 20000, 0.99)
optimizer = tf.keras.optimizers.Adam(learning_rate=decay)
#optimizer = tf.keras.optimizers.RMSprop(learning_rate=decay)
model = NeuralDynamics(6, 2)  # ((24, 6), 2)
EPOCHS = 11

# Training

In [0]:
print(x_states_train.shape, x_actions_train.shape, y_train.shape)
model.compile(optimizer, loss=tf.keras.losses.MeanSquaredError())
model.fit((x_states_train, x_actions_train), y_train, epochs=EPOCHS, batch_size=64)

model.save(join(mounted, 'My Drive/NeuralModel/3/'))

Epoch 1/11
12345/12345 [==============================] - 49s 4ms/step - loss: 533.7521
Epoch 2/11
12345/12345 [==============================] - 49s 4ms/step - loss: 346.9956
Epoch 3/11
12345/12345 [==============================] - 49s 4ms/step - loss: 336.3600
Epoch 4/11
12345/12345 [==============================] - 49s 4ms/step - loss: 325.7378
Epoch 5/11
12345/12345 [==============================] - 49s 4ms/step - loss: 315.7706
Epoch 6/11
12345/12345 [==============================] - 50s 4ms/step - loss: 308.1142
Epoch 7/11
12345/12345 [==============================] - 49s 4ms/step - loss: 299.2908
Epoch 8/11
12345/12345 [==============================] - 49s 4ms/step - loss: 294.1745
Epoch 9/11
12345/12345 [==============================] - 49s 4ms/step - loss: 289.4922
Epoch 10/11
12345/12345 [==============================] - 49s 4ms/step - loss: 285.9679
Epoch 11/11
12345/12345 [==============================] - 49s 4ms/step - loss: 283.5475
INFO:tensorflow:Assets written

# Testing

In [0]:
print(x_states_test.shape, x_actions_test.shape, y_test.shape)
result = model.predict((x_states_test, x_actions_test))

(105343, 6) (105343, 3) (948082, 6)


In [0]:
print(result[0])
print(y_test[0])

[-195.92996   -192.89899      3.0680635   39.463547    10.287734
   -3.7276695]
[-154.736   -174.889      1.29226   19.6261     8.33567   -1.47628]


# Visualization of result

In [0]:
a = np.array((1,2,3))
b = np.array((2,3,4))
print(np.hstack((a,b)), 'h1')

a = np.array([[1],[2],[3]])
b = np.array([[2],[3],[4]])
print(np.hstack((a,b)), 'h2')
print(np.stack((a,b)), 's')
print(np.vstack((a,b)), 'v')
print(zip(a,b), 'z')

[1 2 3 2 3 4] h1
[[1 2]
 [2 3]
 [3 4]] h2
[[[1]
  [2]
  [3]]

 [[2]
  [3]
  [4]]] s
[[1]
 [2]
 [3]
 [2]
 [3]
 [4]] v
<zip object at 0x7f1cd342fa88> z
